#Επιχειρησιακή Έρευνα - Εργασία 2024
####Ονοματεπώνυμο: Οικονόμου Χρήστος
####Α.Ε.Μ.: 10268
####email: cnoikonom@ece.auth.gr

##Εγκατάσταση Βιβλιοθηκών

#####Εισαγωγή του Gurobi Python Module και άλλων χρήσιμων βιβλιοθηκών της Python

In [316]:
%pip install gurobipy

In [317]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB



##1ο Ερώτημα - Προγραμματισμός Προσγειώσεων Πτήσεων

###Εισαγωγή Δεδομένων
#####Ορισμός των δεδομένων εισόδου του μοντέλου

In [318]:
# Earliest time an aircraft can arrive at the airport
earliest_arrival_time = [129, 195, 89, 96, 110, 120, 124, 126, 135, 160]

# Estimated time an aircraft can arrive at the airport
estimated_arrival_time = [155, 258, 96, 106, 123, 135, 138, 140, 150, 180]

# Latest time an aircraft can arrive at the airport
latest_arrival_time = [559, 744, 510, 521, 555, 576, 577, 573, 591, 657]

# Penalties attributed to aircrafts due to early or late arrival at the airport
early_arrival_penalty = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]
late_arrival_penalty = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]

# An array declaring the minimum time required between landings, depending on the aircrafts
min_time_between_landings = [[0, 3, 15, 15, 15, 15, 15, 15, 15, 15],
                             [3, 0, 15, 15, 15, 15, 15, 15, 15, 15],
                             [15, 15, 0, 8, 8, 8, 8, 8, 8, 8],
                             [15, 15, 8, 0, 8, 8, 8, 8, 8, 8],
                             [15, 15, 8, 8, 0, 8, 8, 8, 8, 8],
                             [15, 15, 8, 8, 8, 0, 8, 8, 8, 8],
                             [15, 15, 8, 8, 8, 8, 0, 8, 8, 8],
                             [15, 15, 8, 8, 8, 8, 8, 0, 8, 8],
                             [15, 15, 8, 8, 8, 8, 8, 8, 0, 8],
                             [15, 15, 8, 8, 8, 8, 8, 8, 8, 0]]





###Ανάπτυξη του Μοντέλου
#####Δημιουργία του μοντέλου και ορισμός των μεταβλητών απόφασης

In [319]:
# Model creation
model = gp.Model("aircraft_landing")


# Decision variables declaration
t = model.addVars(10, vtype=GRB.CONTINUOUS, name="t")   # Time of landing of an aircraft
e = model.addVars(10, vtype=GRB.CONTINUOUS, name="e")   # The period of time during which an aircraft lands earlier than the estimated time
l = model.addVars(10, vtype=GRB.CONTINUOUS, name="l")   # The period of time during which an aircraft lands later than the estimated time
z = model.addVars(10, vtype=GRB.BINARY, name="z")       # Indicator binary variables

M = 1000  # A large constant, should be greater than the maximum possible difference


###Ορισμός Αντικειμενικής Συνάρτησης
#####Η αντικειμενική συνάρτηση που περιγράφει το πρόβλημα είναι η:

\begin{equation}
\
\min \sum_{i=1}^{10} \left( p_i^{\text{early}} \cdot e_i + p_i^{\text{late}} \cdot l_i \right)
\
\end{equation}

#####όπου:
- $p_i^{\text{early}}$ είναι η ποινή για πρόωρη προσγείωση του αεροσκάφους $i$.
- $p_i^{\text{late}}$ είναι η ποινή για καθυστερημένη προσγείωση του αεροσκάφους $i$.
- $e_i$ είναι η χρονική διαφορά όταν το αεροσκάφος $i$ προσγειώνεται νωρίτερα από τον εκτιμώμενο χρόνο προσγείωσης (γίνεται ίση με 0 αν το αεροσκάφος προσγειωθεί αργότερα από τον εκτιμώμενο χρόνο προσγείωσης).
- $l_i$ είναι η χρονική διαφορά όταν το αεροσκάφος $i$ προσγειώνεται αργότερα από τον εκτιμώμενο χρόνο προσγείωσης (γίνεται ίση με 0 αν το αεροσκάφος προσγειωθεί νωρίτερα από τον εκτιμώμενο χρόνο προσγείωσης).

\\


#####Η αντικειμενική συνάρτηση εκφράζεται με τον παρακάτω κώδικα:






In [320]:
model.setObjective(gp.quicksum(early_arrival_penalty[i] * e[i] + late_arrival_penalty[i] * l[i] for i in range(10)), GRB.MINIMIZE)

###Περιορισμοί


1. Διασφάλιση ότι οι ώρες προσγείωσης είναι εντός των επιτρεπόμενων χρονικών παραθύρων.
Για κάθε αεροσκάφος $i$:
\begin{align}
t_i &\geq \text{earliest_arrival_time}_i \\
t_i &\leq \text{latest_arrival_time}_i
\end{align}

2. Διασφάλιση σωστού υπολογισμού των ποινών για πρόωρη και καθυστερημένη προσγείωση.
Για κάθε αεροσκάφος $i$:
\begin{align}
e_i &\geq \text{estimated_arrival_time}_i - t_i \\
l_i &\geq t_i - \text{estimated_arrival_time}_i
\end{align}

3. Χρήση του μεγάλου $Μ$ για την επιβολή των λογικών περιορισμών.
Για κάθε αεροσκάφος $i$:
\begin{align}
e_i &\leq M \cdot z_i \\
l_i &\leq M \cdot (1 - z_i)
\end{align}

4. Διασφάλιση μη-αρνητικότητας των ποινών για πρόωρη και καθυστερημένη προσγείωση.
Για κάθε αεροσκάφος $i$:
\begin{align}
e_i &\geq 0 \\
l_i &\geq 0
\end{align}

5. Διασφάλιση του ελάχιστου χρόνου μεταξύ των προσγειώσεων.
Για κάθε ζεύγος αεροσκαφών $i$ και $j$ με $i < j$:
\begin{align}
t_j &\geq t_i + \text{min_time_between_landings}_{i,j}
\end{align}




In [321]:
# Ensure landing times are within the allowed time window
for i in range(10):
    model.addConstr(t[i] >= earliest_arrival_time[i], f"earliest_{i}")
    model.addConstr(t[i] <= latest_arrival_time[i], f"latest_{i}")

# Ensure correct calculation of early and late landings
for i in range(10):
    model.addConstr(e[i] >= estimated_arrival_time[i] - t[i], f"early_{i}")
    model.addConstr(l[i] >= t[i] - estimated_arrival_time[i], f"late_{i}")

    # Use big-M to enforce the logical constraints
    model.addConstr(e[i] <= M * z[i], f"bigM1_{i}")
    model.addConstr(l[i] <= M * (1 - z[i]), f"bigM2_{i}")

    # Ensure non-negativity of early and late landing times
    model.addConstr(e[i] >= 0, f"nonneg_e_{i}")
    model.addConstr(l[i] >= 0, f"nonneg_l_{i}")

# Ensure minimum time between landings
for i in range(10):
    for j in range(i + 1, 10):
        model.addConstr(t[j] >= t[i] + min_time_between_landings[i][j], f"separation1_{i}_{j}")

###Βελτιστοποίηση
#####Εύρεση της βέλτιστης λύσης μέσω του Gurobi Solver

In [322]:
model.optimize()

# If the model is infeasible, use Gurobi's infeasibility diagnostic tools
if model.status == GRB.INFEASIBLE:
    print('Model is infeasible')
    model.computeIIS()
    model.write("model.ilp")

    # Read and print the IIS file to understand the conflicts
    with open("model.ilp", "r") as f:
        for line in f:
            print(line.strip())

# Print results if feasible
if model.status == GRB.OPTIMAL:
    for i in range(10):
        print(f"Aircraft {i+1} lands at time {t[i].X}")
        print(f"  Early landing penalty time: {e[i].X}")
        print(f"  Late landing penalty time: {l[i].X}")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 125 rows, 40 columns and 210 nonzeros
Model fingerprint: 0x66f649c1
Variable types: 30 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+03]
Presolve removed 125 rows and 40 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 25710 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.571000000000e+04, best bound 2.571000000000e+04, gap 0.0000%
Aircraft 1 lands at time 155.0
  Early landing penalty time: 0.0
  Late landing penalty t

###Αποτελέσματα
#####Για τα δεδομένα που παρουσιάστηκαν, εκτιμήθηκε πως το ελάχιστο συνολικό κόστος ανέρχεται στα 25710, ενώ ο βέλτιστος προγραμματισμός που προέκυψε έχει ως εξής:
| Αεροσκάφος | Χρόνος Προσγείωσης | Πρόστιμο Πρόωρης Προσγείωσης | Πρόστιμο Καθυστερημένης Προσγείωσης |
| --- | --- | --- | --- |
| 1 | 155 | 0 | 0 |
| 2 | 195 | 63 | 0 |
| 3 | 210 | 0 | 114 |
| 4 | 218 | 0 | 112 |
| 5 | 226 | 0 | 103 |
| 6 | 234 | 0 | 99 |
| 7 | 242 | 0 | 104 |
| 8 | 250 | 0 | 110 |
| 9 | 258 | 0 | 108 |
| 10 | 266 | 0 | 86 |








  

##2ο Ερώτημα - Σχεδιασμός Παράδοσης Πετρελαίου

###Εισαγωγή Δεδομένων
#####Ορισμός των δεδομένων εισόδου του μοντέλου

In [323]:
# Points of the refinery and the cities
points = ['Ω', 'Α', 'Β', 'Γ', 'Δ', 'Ε', 'ΣΤ']

# Oil demand for each city (in liters)
oil_demand = [14000, 3000, 6000, 16000, 15000, 5000]

# Array of distances between the refinery and the cities in demand (in kilometers)
distances = [[0, 148, 55, 32, 70, 140, 73],
             [148, 0, 93, 180, 99, 12, 72],
             [55, 93, 0, 85, 20, 83, 28],
             [32, 180, 85, 0, 100, 174, 99],
             [70, 99, 20, 100, 0, 85, 49],
             [140, 12, 83, 174, 85, 0, 73],
             [73, 72, 28, 99, 49, 73, 0]]

# Storage capacity of the tankers (in liters)
storage_capacity = 39000

# Convert distances to a dictionary for ease of use with Gurobi
dist_dict = {}
for i in range(len(points)):
    for j in range(len(points)):
        dist_dict[(points[i], points[j])] = distances[i][j]

###Ανάπτυξη του Μοντέλου
#####Δημιουργία του μοντέλου και ορισμός των μεταβλητών απόφασης

In [324]:
# Model creation
model = gp.Model('OilDelivery')

# Decision variables declaration
x = model.addVars(dist_dict.keys(), vtype=GRB.BINARY, name="x")   # Binary variables that show if a route between two points is used
u = model.addVars(points[1:], vtype=GRB.CONTINUOUS, name="u")     # Continuous variables for sub-tour balancing

###Ορισμός Αντικειμενικής Συνάρτησης
#####Η αντικειμενική συνάρτηση που περιγράφει το πρόβλημα είναι η:

\begin{equation}
\
\min \sum_{(i, j) \in \text{dist_dict.keys()}} \text{dist_dict}[i, j] \cdot x[i, j]
\
\end{equation}

#####όπου:
- $dist\_dict[i,j]$ είναι η απόσταση μεταξύ των πόλεων $i$ και $j$.
- $x[i,j]$ είναι η δυαδική μεταβλητή που παίρνει την τιμή 1 αν η διαδρομή από την πόλη $i$ στην πόλη $j$ χρησιμοποιείται, αλλιώς παίρνει την τιμή 0.


\\
#####Η αντικειμενική συνάρτηση εκφράζεται με τον παρακάτω κώδικα:


In [325]:
model.setObjective(gp.quicksum(dist_dict[i, j] * x[i, j] for i, j in dist_dict.keys()), GRB.MINIMIZE)


###Περιορισμοί

1.  Κάθε σημείο (εκτός από το διυλιστήριο) πρέπει να δέχεται επίσκεψη μόνο μία φορά:

  Για κάθε σημείο $j$ εκτός από το διυλιστήριο, η ποσότητα των εισερχομένων διαδρομών πρέπει να είναι ίση με την ποσότητα των εξερχομένων διαδρομών, και αυτές πρέπει να είναι ίσες με 1.
  
\begin{equation}
\sum_{i \in \text{points}} x[i, j] = 1, \quad \forall \ j \neq Ω
\end{equation}

\begin{equation}
\sum_{j \in \text{points}} x[j, k] = 1, \quad \forall \ k \neq Ω
\end{equation}

2.  Περιορισμοί ισορροπίας υπο-περιοδείας:

  Για κάθε σημεία $i$ και $j$, όπου $i \neq j$ και υπάρχει απόσταση μεταξύ τους, ελέγχουμε ότι η ποσότητα πετρελαίου που φεύγει από το σημείο $i$ πρέπει να είναι μικρότερη ή ίση από τη χωρητικότητα του τανκερ μείον την ποσότητα πετρελαίου που φτάνει στο σημείο $j$.

\begin{equation}
u[i] - u[j] + \text{storage_capacity} \cdot x[i, j] \leq \text{storage_capacity} - \text{oil_demand}[j-1], \quad \forall \ i, j \neq Ω \ \text{και} \ (i, j) \in \text{dist_dict}
\end{equation}

\\
3.  Το πετρελαιοφόρο πρέπει να μεταφέρει αρκετή ποσότητα πετρελαίου για να καλύψει τη ζήτηση:

  Για κάθε σημείο $i$ εκτός από το διυλιστήριο, η ποσότητα πετρελαίου που φεύγει από αυτό πρέπει να είναι τουλάχιστον ίση με τη ζήτηση πετρελαίου του αντίστοιχου σημείου.

\begin{equation}
u[i] \geq \text{oil_demand}[i-1], \quad \forall \ i \neq Ω
\end{equation}

4.  Το πετρελαιοφόρο μπορεί να μεταφέρει έως 39000 λίτρα ανά διαδρομή:

  Για κάθε διαδρομή από την πόλη $i$ στην πόλη $j$, η ποσότητα του πετρελαίου που μεταφέρεται δεν πρέπει να υπερβαίνει τη χωρητικότητα του τάνκερ.

\begin{equation}
x[i, j] \cdot \text{oil_demand}[j-1] \leq \text{storage_capacity}, \quad \forall \ (i, j) \in \text{dist_dict}
\end{equation}

5.  Αποφυγή επιστροφής στο ίδιο σημείο:

  Η ποσότητα πετρελαίου που μεταφέρεται από το σημείο $i$ στο ίδιο σημείο $i$ πρέπει να είναι μηδέν.

\begin{equation}
x[i, i] = 0, \quad \forall \ i
\end{equation}


In [326]:
# Every point (except for the refinery) must be visited only once
for j in points:
    if j != 'Ω':
        model.addConstr(gp.quicksum(x[i, j] for i in points if (i, j) in dist_dict) == 1)
        model.addConstr(gp.quicksum(x[j, k] for k in points if (j, k) in dist_dict) == 1)

# Sub-tour balancing constraints
for i in points[1:]:
    for j in points[1:]:
        if i != j and (i, j) in dist_dict:
            model.addConstr(u[i] - u[j] + storage_capacity * x[i, j] <= storage_capacity - oil_demand[points.index(j) - 1])

# The tanker needs to carry a sufficient amount of oil, to cover the demand
for i in points[1:]:
    model.addConstr(u[i] >= oil_demand[points.index(i) - 1])

# The tanker can only carry up to 39000 liters per route
for i, j in dist_dict.keys():
    model.addConstr(x[i, j] * oil_demand[points.index(j) - 1] <= storage_capacity)

# Avoid returning to the same point
for i in points:
    model.addConstr(x[i, i] == 0)

###Βελτιστοποίηση
#####Εύρεση της βέλτιστης λύσης μέσω του Gurobi Solver

In [327]:
model.optimize()

# Printing results
if model.status == GRB.OPTIMAL:
    solution = model.getAttr('x', x)
    selected_routes = {k: v for k, v in solution.items() if v > 0.5}

    print('Βέλτιστες διαδρομές:')

    def find_next_city(current_city):
        for (i, j), value in selected_routes.items():
            if i == current_city:
                return j
        return None

    # Function to find all routes starting from Ω
    def find_routes_from_Ω():
        routes = []
        for (i, j), value in selected_routes.items():
            if i == 'Ω':
                route = ['Ω', j]
                next_city = j
                while next_city != 'Ω':
                    next_city = find_next_city(next_city)
                    if next_city is not None:
                        route.append(next_city)
                    else:
                        break
                route.append('Ω')
                routes.append(route)
        return routes

    # Find all routes starting from Ω
    routes_from_Ω = find_routes_from_Ω()

    # Print all routes starting from Ω
    for route in routes_from_Ω:
        for i in range(len(route) - 1):
            print(f'{route[i]} -> {route[i + 1]}')
else:
    print('Δεν βρέθηκε βέλτιστη λύση')


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 104 rows, 55 columns and 236 nonzeros
Model fingerprint: 0x6c0e53aa
Variable types: 6 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Found heuristic solution: objective 826.0000000
Presolve removed 62 rows and 23 columns
Presolve time: 0.00s
Presolved: 42 rows, 32 columns, 138 nonzeros
Variable types: 6 continuous, 26 integer (26 binary)
Found heuristic solution: objective 515.0000000

Root relaxation: objective 2.461282e+02, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

###Αποτελέσματα
#####Με βάση την παραπάνω ανάλυση, προκύπτει πως η βέλτιστη διαδρομή είναι:
\
\begin{align*}
Ω &\rightarrow Β \rightarrow Ε \rightarrow Α \rightarrow ΣΤ \rightarrow Ω \rightarrow Γ \rightarrow Δ \rightarrow Ω \\
\end{align*}

##### και έχει συνολικό μήκος 497 χιλιόμετρα.
